<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#데이터-로드-->-날짜,-자기상관-변수-FE-함수화" data-toc-modified-id="데이터-로드-->-날짜,-자기상관-변수-FE-함수화-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>데이터 로드 -&gt; 날짜, 자기상관 변수 FE 함수화</a></span></li><li><span><a href="#첫-번째-제출:-1년-전-변수까지만-포함해서" data-toc-modified-id="첫-번째-제출:-1년-전-변수까지만-포함해서-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>첫 번째 제출: 1년 전 변수까지만 포함해서</a></span></li><li><span><a href="#두-번째-제출:-기상-데이터-~2018-12-31년것까지-가져와서-사용-후-제출" data-toc-modified-id="두-번째-제출:-기상-데이터-~2018-12-31년것까지-가져와서-사용-후-제출-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>두 번째 제출: 기상 데이터 ~2018-12-31년것까지 가져와서 사용 후 제출</a></span></li><li><span><a href="#버전-2-결론" data-toc-modified-id="버전-2-결론-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>버전 2 결론</a></span></li></ul></div>

## 데이터 로드 -> 날짜, 자기상관 변수 FE 함수화

In [105]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import pymysql
import warnings
warnings.filterwarnings(action='ignore')


def load_datasets(path) -> pd.DataFrame:
    train = pd.read_csv(os.path.join(path, '한국가스공사_시간별 공급량_20181231.csv'),
                        encoding='cp949')
    test = pd.read_csv(os.path.join(path, 'test.csv'))
    # test 데이터 전처리
    test['연월일'] = test['일자|시간|구분'].str.split(' ', expand=True)[0]
    test['시간'] = test['일자|시간|구분'].str.split(' ', expand=True)[1].astype(int)
    test['구분'] = test['일자|시간|구분'].str.split(' ', expand=True)[2]
    del test['일자|시간|구분']

    data = pd.concat([train, test], axis=0)
    data['연월일'] = pd.to_datetime(data['연월일'])
    return data

def load_weather() -> pd.DataFrame:
    db = pymysql.connect(host='localhost', port=3306, user='younghun', password='watson1259',
                        db='dacon_gas_weather_db', charset='utf8')
    cursor = db.cursor()
    sql = "SELECT datetime, avg_temp, min_temp, max_temp,\
              NULLIF(sum_rain, '') as sum_rain, avg_wind, avg_humid,\
              sum_gsr, NULLIF(ddmefs, '') as ddmefs, avg_ts \
              FROM weather ORDER BY datetime"
    
    weather = pd.read_sql(sql, db)
    weather = weather.fillna(0.)
    
    return weather
    
def merge_gas_weather(path):
    gas = load_datasets(path)
    weather = load_weather()
    gas_weather = gas.merge(weather, how='left', left_on='연월일', right_on='datetime')
    del gas_weather['datetime']
    
    return gas_weather

def make_datetime_vars(data) -> pd.DataFrame:
    data['year'] = data['연월일'].dt.year
    data['month'] = data['연월일'].dt.month
    data['day'] = data['연월일'].dt.day
    data['week_no'] = data['연월일'].dt.strftime("%V").astype(int)
    data['dayofweek'] = data['연월일'].dt.day_name()
    data['weekend_yn'] = np.where(data['dayofweek'].isin(['Saturday', 'Sunday']), 1, 0)
    data['dayofyear'] = data['연월일'].dt.dayofyear

    return data


def change_dates(row):
    if row['시간'] == 24:
        row['연월일'] += pd.DateOffset(days=1)
        row['시간'] = 0
    return row


def extract_need_data(df):
    """ 연월일 날짜 포맷 바꾸고 매년 01-01~03-31 기간 데이터만 추출"""
    df = df.apply(change_dates, axis=1)
    df['시간'] = df['시간'].apply(lambda x: str(x) if x >= 10 else '0' + str(x))
    df['연월일'] = df['연월일'].astype(str)
    df['datetime'] = df['연월일'] + ' ' + df['시간']
    df['datetime'] = pd.to_datetime(df['datetime'], format='%Y-%m-%d %H')

    df['prev_datetime'] = df['datetime'] - pd.DateOffset(years=1)

    return df


def make_autocorr_vars(df, gubun='A'):
    sub_df = df[df['구분'] == gubun]
    # 일년 전 그 시간대 공급량
    sub_df['prev_year_공급량'] = sub_df.groupby([sub_df['datetime'].dt.month, sub_df['datetime'].dt.day,
                                          sub_df['datetime'].dt.hour])['공급량'].shift()
#     # 이년 전 공급량 파생변수 생성
#     sub_df['prev_two_year_공급량'] = sub_df.groupby([sub_df['datetime'].dt.month, sub_df['datetime'].dt.day,
#                                               sub_df['datetime'].dt.hour])['공급량'].shift(2)
    # 일년 전 평균 공급량
    sub_df['prev_year'] = sub_df['prev_datetime'].dt.year
    prev_year_mean_df = sub_df.groupby([sub_df['datetime'].dt.year])['공급량'].mean()
    prev_year_mean_dict = dict(zip(prev_year_mean_df.index, prev_year_mean_df.values.reshape(-1, )))
    sub_df['prev_year_avg_공급량'] = sub_df['prev_year'].map(prev_year_mean_dict)
    # 일년 전 월별 평균 공급량
    prev_month_mean_df = sub_df.groupby([sub_df['datetime'].dt.year, sub_df['datetime'].dt.month])['공급량'].mean()
    prev_month_mean_dict = dict(zip(prev_month_mean_df.index, prev_month_mean_df.values.reshape(-1, )))
    sub_df['prev_year_month'] = tuple(zip(sub_df['prev_year'].values.reshape(-1, ),
                                        sub_df['prev_datetime'].dt.month.values.reshape(-1, )))
    sub_df['prev_month_avg_공급량'] = sub_df['prev_year_month'].map(prev_month_mean_dict)
    # 일년 전 월-일자별 평균 공급량
    prev_month_day_mean_df = sub_df.groupby([sub_df['datetime'].dt.year,
                                           sub_df['datetime'].dt.month,
                                           sub_df['datetime'].dt.day])['공급량'].mean()
    prev_month_day_mean_dict = dict(zip(prev_month_day_mean_df.index, prev_month_day_mean_df.values.reshape(-1, )))

    sub_df['prev_year_month_day'] = tuple(zip(sub_df['prev_year'].values.reshape(-1, ),
                                            sub_df['prev_datetime'].dt.month.values.reshape(-1, ),
                                            sub_df['prev_datetime'].dt.day.values.reshape(-1, )))
    sub_df['prev_month_day_avg_공급량'] = sub_df['prev_year_month_day'].map(prev_month_day_mean_dict)
    # 일년 전 월-시간 별 평균 공급량
    prev_month_hour_mean_df = sub_df.groupby([sub_df['datetime'].dt.year,
                                            sub_df['datetime'].dt.month,
                                            sub_df['datetime'].dt.hour])['공급량'].mean()
    prev_month_hour_mean_dict = dict(zip(prev_month_hour_mean_df.index, prev_month_hour_mean_df.values.reshape(-1, )))

    sub_df['prev_year_month_hour'] = tuple(zip(sub_df['prev_year'].values.reshape(-1, ),
                                             sub_df['prev_datetime'].dt.month.values.reshape(-1, ),
                                             sub_df['prev_datetime'].dt.hour.values.reshape(-1, )))
    sub_df['prev_month_hour_avg_공급량'] = sub_df['prev_year_month_hour'].map(prev_month_hour_mean_dict)
    # 일년 전 그 날의 기상 변수 FE
    weather_cols = ['avg_temp','min_temp','max_temp','sum_rain','avg_wind','avg_humid','sum_gsr','ddmefs','avg_ts']
    for col in weather_cols:
        sub_df[f'prev_year_{col}'] = sub_df.groupby([sub_df['datetime'].dt.month, sub_df['datetime'].dt.day,
                                                  sub_df['datetime'].dt.hour])[col].shift()
        sub_df[f'prev_year_{col}'] = pd.to_numeric(sub_df[f'prev_year_{col}'], downcast="float")

    # 필요한 칼럼들만 추출
#     used_cols = ['연월일', '시간', 'datetime', '구분', 'prev_year_공급량',
#                  'prev_two_year_공급량', 'prev_year_avg_공급량', 'prev_month_avg_공급량', 'prev_month_day_avg_공급량',
#                  'prev_month_hour_avg_공급량', '공급량']
    used_cols = ['연월일', '시간', 'datetime', '구분', 'prev_year_공급량', 'prev_year_avg_공급량',
                 'prev_month_avg_공급량', 'prev_month_day_avg_공급량', 'prev_month_hour_avg_공급량']
    weather_cols = [f'prev_year_{col}' for col in weather_cols]
    used_cols += weather_cols + ['공급량']
    sub_df = sub_df[used_cols]
    return sub_df


def fe_autocorr_vars(df):
    final_df = pd.DataFrame()
    gubun_cols = df['구분'].unique()
    for gubun in tqdm(gubun_cols):
        sub_df = make_autocorr_vars(df, gubun=gubun)
        final_df = pd.concat([final_df, sub_df], axis=0)
    return final_df

def change_dates_adversely(row):
    if row['datetime'].hour == 0:
        row['연월일'] -= pd.DateOffset(days=1)
        row['시간'] = 24
    return row

def change_date_format(df):
    df['연월일'] = pd.to_datetime(df['연월일'])
    df['시간'] = df['시간'].astype(int)
    df = df.apply(change_dates_adversely, axis=1)
    
    del df['datetime']
    
    return df


path = '/Users/younghun/Desktop/gitrepo/dacon_gas'
gas_weather = merge_gas_weather(path)
dataset = make_datetime_vars(gas_weather)
dataset = extract_need_data(dataset)
final_df = fe_autocorr_vars(dataset)
final_df = change_date_format(final_df)
final_df = final_df.reset_index(drop=True)
print(final_df.shape)




100%|██████████| 7/7 [00:06<00:00,  1.05it/s]


(383208, 18)


## 첫 번째 제출: 1년 전 변수까지만 포함해서
- Train: 2014-01-01~
- Train NMAE: 
- Valid NMAE: 

In [161]:
df = final_df.copy()
print('총 데이터프레임:', df.shape)

train = df[df['연월일'].dt.year < 2019]
test = df[df['연월일'].dt.year >= 2019]
print('Train:', train.shape)
print('Test:', test.shape)

if train.shape[0] + test.shape[0] == df.shape[0]:
    print('데이터 개수 맞음')
else:
    print('데이터 개수 틀림')

총 데이터프레임: (383208, 9)
Train: (368088, 9)
Test: (15120, 9)
데이터 개수 맞음


In [162]:
# Train: 2014년 부터 사용하고 01-01 ~ 03-31 기간만 추출
train_cond1 = (train['연월일'].dt.year >= 2014)
train_cond2 = (train['연월일'].dt.month >= 1) & (train['연월일'].dt.month <= 3)
train = train[train_cond1]
train = train[train_cond2]
print(train.shape)

(75768, 9)


In [163]:
# '구분' 칼럼에 대하여 원-핫 인코딩 수행
ohe_cols = ['B', 'C', 'D', 'E', 'G', 'H']

train[ohe_cols] = pd.get_dummies(train['구분'], drop_first=True)
test[ohe_cols] = pd.get_dummies(test['구분'], drop_first=True)


# 칼럼 순서 배열 재배치
cols_order = train.columns.tolist()
cols_order.remove('공급량')
cols_order.insert(len(cols_order), '공급량')

train = train[cols_order]
test = test[cols_order]

In [164]:
# 멀티인덱스 설정
train = train.set_index(['연월일', '시간', '구분'])
test = test.set_index(['연월일', '시간', '구분'])

In [172]:
# 바로 모델링 진행 -> LGBM
from lightgbm import LGBMRegressor

# 마지막 2018-12-31 데이터 제외
idx_level = train.index.get_level_values
# Train/Valid 분할
train_df = train[(idx_level(0) < '2018-01-01')]
valid_df = train[(idx_level(0) >= '2018-01-01') & (idx_level(0) <= '2018-03-31')]
print('Train:', train_df.shape)
print('Valid:', valid_df.shape)


# X, y 분할
X_train, y_train = train_df.values[:, :-1], train_df['공급량']
X_valid, y_valid = valid_df.values[:, :-1], valid_df['공급량']

# model
model = LGBMRegressor(n_estimators=100, min_child_samples=20)

# fit
model.fit(X_train, y_train)

# predict
train_pred = model.predict(X_train)
valid_pred = model.predict(X_valid)

# evaluate
train_NMAE = np.mean((np.abs(y_train-train_pred))/y_train)
valid_NMAE = np.mean((np.abs(y_valid-valid_pred))/y_valid)

print('Train NMAE:', train_NMAE)
print('Valid NMAE:', valid_NMAE)

Train: (60648, 12)
Valid: (15120, 12)
Train NMAE: 0.11323516981132432
Valid NMAE: 0.16725128664449224


In [177]:
# Test 데이터에 대해 최종 예측

X_train, y_train = train.iloc[:, :-1].values, train['공급량']
X_test = test.iloc[:, :-1].values

# fit
model = LGBMRegressor(n_estimators=100)
model.fit(X_train, y_train)

# predict
test_pred = model.predict(X_test)
print('test_pred shape:', test_pred.shape)

test_pred shape: (15120,)


In [179]:
sample_sub['공급량'] = test_pred

In [187]:
sample_sub.to_csv('/Users/younghun/Desktop/gitrepo/dacon_gas/submission/lgbm_base01.csv', index=False)

## 두 번째 제출: 기상 데이터 ~2018-12-31년것까지 가져와서 사용 후 제출

In [58]:
# import requests
# from bs4 import BeautifulSoup
# import pymysql

# time_delta = pd.Timestamp('2018-12-31') - pd.Timestamp('2013-01-01')
# time_delta = time_delta.days + 1
# max_pages = math.ceil((time_delta / 999) + 1)

# for page in range(1, max_pages):
#     # 서울 기상 데이터만 추출하는 예시
#     key = 'j%2FJXmL%2BFxwnNYqN%2FyoLJSfJx3ioQV1HnmM9E7b%2FaGLjLv51g0vZSGQjk0UVyJmGZckzK7Cm8Jds6G42cqdkX0w%3D%3D'
#     start_dt = '20130101'
#     end_dt = '20181231'
#     url = f'http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList?serviceKey={key}&numOfRows=999&pageNo={page}&dataCd=ASOS&dateCd=DAY&startDt={start_dt}&endDt={end_dt}&stnIds=108'
    
#     res = requests.get(url)
#     soup = BeautifulSoup(res.content, 'html.parser')
    
#     data_dict = {}
    
#     region, datetime = [], []
#     avg_temp, min_temp = [], []
#     max_temp, sum_rain = [], []
#     avg_wind, avg_humid = [], []
#     sum_gsr, ddmefs = [], []
#     avg_ts = []
    
#     contents = soup.select('item')
#     for content in contents:
#         reg = content.select_one('stnnm').get_text()
#         date = content.select_one('tm').get_text()
#         avg_t = content.select_one('avgta').get_text()
#         min_t = content.select_one('minta').get_text()
#         max_t = content.select_one('maxta').get_text()
#         sum_r = content.select_one('sumrn').get_text()
#         avg_w = content.select_one('avgws').get_text()
#         avg_h = content.select_one('avgrhm').get_text()
#         sum_g = content.select_one('sumgsr').get_text()
#         ddme = content.select_one('ddmefs').get_text()
#         avg_s = content.select_one('avgts').get_text()
        
#         region.append(reg)
#         datetime.append(date)
#         avg_temp.append(avg_t)
#         min_temp.append(min_t)
#         max_temp.append(max_t)
#         sum_rain.append(sum_r)
#         avg_wind.append(avg_w)
#         avg_humid.append(avg_h)
#         sum_gsr.append(sum_g)
#         ddmefs.append(ddme)
#         avg_ts.append(avg_s)
    
#     data_dict['지역'] = region
#     data_dict['시간'] = datetime
#     data_dict['평균기온'] = avg_temp
#     data_dict['최저기온'] = min_temp
#     data_dict['최고기온'] = max_temp
#     data_dict['합계강수량'] = sum_rain
#     data_dict['평균풍속'] = avg_wind
#     data_dict['평균습도'] = avg_humid
#     data_dict['합계일사량'] = sum_gsr
#     data_dict['신적설'] = ddmefs
#     data_dict['평균지면온도'] = avg_ts   
    
#     db = pymysql.connect(host='localhost', port=3306, user='younghun', password='watson1259',
#                         db='dacon_gas_weather_db', charset='utf8')
#     cursor = db.cursor()
#     rows = map(list, zip(*data_dict.values()))
#     for row in rows:
#         sql = """INSERT INTO weather VALUES(
#         '"""+row[0]+"""','"""+row[1]+"""','"""+row[2]+"""','"""+row[3]+"""','"""+row[4]+"""',
#         '"""+row[5]+"""','"""+row[6]+"""','"""+row[7]+"""','"""+row[8]+"""','"""+row[9]+"""',
#         '"""+row[10]+"""')"""
#         cursor.execute(sql)
#         db.commit()
#     db.close()

In [133]:
df = final_df.copy()
print('총 데이터프레임:', df.shape)

train = df[df['연월일'].dt.year < 2019]
test = df[df['연월일'].dt.year >= 2019]
print('Train:', train.shape)
print('Test:', test.shape)

if train.shape[0] + test.shape[0] == df.shape[0]:
    print('데이터 개수 맞음')
else:
    print('데이터 개수 틀림')

총 데이터프레임: (383208, 18)
Train: (368088, 18)
Test: (15120, 18)
데이터 개수 맞음


In [127]:
# Train: 2014년 부터 사용하고 01-01 ~ 03-31 기간만 추출
train_cond1 = (train['연월일'].dt.year >= 2014)
train_cond2 = (train['연월일'].dt.month >= 1) & (train['연월일'].dt.month <= 3)
train = train[train_cond1]
train = train[train_cond2]
print(train.shape)

(75768, 18)


In [134]:
# '구분' 칼럼에 대하여 원-핫 인코딩 수행
ohe_cols = ['B', 'C', 'D', 'E', 'G', 'H']

train[ohe_cols] = pd.get_dummies(train['구분'], drop_first=True)
test[ohe_cols] = pd.get_dummies(test['구분'], drop_first=True)


# 칼럼 순서 배열 재배치
cols_order = train.columns.tolist()
cols_order.remove('공급량')
cols_order.insert(len(cols_order), '공급량')

train = train[cols_order]
test = test[cols_order]

In [135]:
# 멀티인덱스 설정
train = train.set_index(['연월일', '시간', '구분'])
test = test.set_index(['연월일', '시간', '구분'])

In [137]:
# 바로 모델링 진행 -> LGBM
from lightgbm import LGBMRegressor

# 마지막 2018-12-31 데이터 제외
idx_level = train.index.get_level_values
# Train/Valid 분할
train_df = train[(idx_level(0) < '2018-01-01')]
valid_df = train[(idx_level(0) >= '2018-01-01') & (idx_level(0) <= '2018-03-31')]
print('Train:', train_df.shape)
print('Valid:', valid_df.shape)


# X, y 분할
X_train, y_train = train_df.values[:, :-1], train_df['공급량']
X_valid, y_valid = valid_df.values[:, :-1], valid_df['공급량']

# model
model = LGBMRegressor(n_estimators=100, min_child_samples=20)

# fit
model.fit(X_train, y_train)

# predict
train_pred = model.predict(X_train)
valid_pred = model.predict(X_valid)

# evaluate
train_NMAE = np.mean((np.abs(y_train-train_pred))/y_train)
valid_NMAE = np.mean((np.abs(y_valid-valid_pred))/y_valid)

print('Train NMAE:', train_NMAE)
print('Valid NMAE:', valid_NMAE)

Train: (306768, 21)
Valid: (15120, 21)
Train NMAE: 0.8123844816204941
Valid NMAE: 0.16915868159337155


In [138]:
# Test 데이터로 최종 예측
X_train, y_train = train.values[:, :-1], train['공급량']
X_test = test.values[:, :-1]

model = LGBMRegressor(n_estimators=100, min_child_samples=20)

model.fit(X_train, y_train)

# predict
test_pred = model.predict(X_test)
print(test_pred.shape)  # 15,120개여야 함!

(15120,)


In [139]:
sample_csv = pd.read_csv('/Users/younghun/Desktop/gitrepo/dacon_gas/sample_submission.csv')
sample_csv['공급량'] = test_pred
sample_csv.to_csv('/Users/younghun/Desktop/gitrepo/dacon_gas/submission/lgbm_base_weather(all_data).csv',
                 index=False) # 모든 데이터로 예측

## 버전 2 결론
- 날씨 데이터 사용의 효과가 그닥.. -> 날씨 데이터 그만파자.. 오히려 빼는게 좋을 수도.. -> 자기상관변수 증감률 데이터 FE하기!
- 모든 데이터를 다 사용했을 때가 Test 데이터에 대해 리더보드 결과는 더 좋음.. -> 모든 데이터 다 사용하기
- 다른 모델들 사용 -> 여러가지 모델...뭘 사용할까!?